In [30]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import time

video_path = '/content/drive/MyDrive/DR_AI_응용/data/newyork.mp4'

In [31]:
"""
generate instance, varaibles(line,criterion,frame, optical flow)
copy frame(presever original image)
gray scale
trace(previous image none-> fist image, otherwise optical flow)
p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, gray, p0, None,
                                      winSize=(15,15),
                                      maxLevel=2,
                                      criteria=termcriteria)

p1 : position of new trace
st : success or fail
err : error of trace
old_gray, gray : previous and current frame
p0 : the position where we want to trce
None :
winsize= window size to trace (large- noise good, variation no | small-trace small object, harsh movement)
maxelvel(pyramid): until which level(reduced size) we weill trace(because they might loose big movenement, cuz it is specialized to dectect small movement )
critereion : tolerance to error
"""

'\ngenerate instance, varaibles(line,criterion,frame, optical flow)\ncopy frame(presever original image)\ngray scale\ntrace(previous image none-> fist image, otherwise optical flow)\np1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, gray, p0, None,\n                                      winSize=(15,15),\n                                      maxLevel=2,\n                                      criteria=termcriteria)\n\np1 : position of new trace\nst : success or fail\nerr : error of trace\nold_gray, gray : previous and current frame\np0 : the position where we want to trce\nNone : \nwinsize= window size to trace (large- noise good, variation no | small-trace small object, harsh movement)\nmaxelvel(pyramid): until which level(reduced size) we weill trace(because they might loose big movenement, cuz it is specialized to dectect small movement )\ncritereion : tolerance to error\n'

In [ ]:
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
  print("can't open video")
  exit()

MAX_FRAME = 200
CAPTUR_EVERY_FRAME = 20
frame_count = 0
colors = np.random.randint(0,255,(200,3))

lines = None
preimg = None
criterion = (cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_COUNT,10,0.1) # maximum count = 10, maximum eps 0.03


while cap.isOpened() and frame_count < MAX_FRAME :
  ret, frame = cap.read()
  if not ret :
    break

  img_draw = frame.copy()
  gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

  if preimg is None:
    preimg = gray
    lines = np.zeros_like(frame)
    prevPt = cv2.goodFeaturesToTrack(preimg,50,0.01,10)

  else :
    crrimg = gray
    nextPt,st,err = cv2.calcOpticalFlowPyrLK(preimg, crrimg, prevPt,None,criteria=criterion)
    prevMv = prevPt[st == 1]
    nextMv = nextPt[st == 1]

    for i,(p,n) in enumerate(zip(prevMv,nextMv)):
      px,py = p.ravel()
      nx, ny = n.ravel()

      cv2.line(lines, (int(px),int(py)), (int(nx),int(ny)), colors[i % len(colors)].tolist(),2)
      cv2.circle(img_draw,(int(nx),int(ny)),8,colors[i%len(colors)].tolist(),-1)

    img_draw = cv2.add(img_draw,lines)
    preimg = crrimg
    prevPt = nextMv.reshape(-1,1,2)


  if frame_count % CAPTUR_EVERY_FRAME == 0 :
        print(f"\n--- Frame {frame_count}---")
        cv2_imshow(img_draw)
        time.sleep(1)


  frame_count +=1

cap.release()

